# Find Peak Load Days in Each Month

In [42]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#create temporary DF to find peak
test = pd.read_csv('outputs/load_long_format.csv')

#groupby region, month, and day to sum the total day load
aggregations1 = {'Load':sum}
test_sum = test.groupby(['Region','Month','Day'],as_index=False).agg(aggregations1)
#test1.columns = test1.columns.droplevel(0)
test_sum.columns = ['Region','Month','Day','Load_Tot']
#print(test_sum.head())
#print('number of rows in dataset =',test_sum.shape[0])

test3 = pd.merge(test,test_sum,on=['Region','Month','Day'],how='left')
#print(test3)

#groupby region and month to find maximum load
aggregations2 = {'Load_Tot':max}
test_max = test_sum.groupby(['Region','Month'],as_index=False).agg(aggregations2)
test_max.columns = ['Region','Month','Load_Max']
#print(test_max.head())

test4 = pd.merge(test3,test_max,on=['Region','Month'],how='left')
test4 = test4.drop(test4.columns[0], axis=1)
print(test4)

          Region R_Group R_Subgroup  Season  Month  Day  Hour  Hour_Counter  \
0       ERC_REST     ERC       REST  winter      1    1     1             1   
1       ERC_REST     ERC       REST  winter      1    1     2             2   
2       ERC_REST     ERC       REST  winter      1    1     3             3   
3       ERC_REST     ERC       REST  winter      1    1     4             4   
4       ERC_REST     ERC       REST  winter      1    1     5             5   
...          ...     ...        ...     ...    ...  ...   ...           ...   
551875  WEC_SDGE     WEC       SDGE  winter     12  365    20          8756   
551876  WEC_SDGE     WEC       SDGE  winter     12  365    21          8757   
551877  WEC_SDGE     WEC       SDGE  winter     12  365    22          8758   
551878  WEC_SDGE     WEC       SDGE  winter     12  365    23          8759   
551879  WEC_SDGE     WEC       SDGE  winter     12  365    24          8760   

         Load  Load_Tot  Load_Max  
0       34807  

In [52]:
load_dur1 = pd.read_csv('outputs/load_long_format.csv', index_col=0)
load_years = pd.read_csv('inputs/load_years.csv').dropna()
#print(load_dur1.head())
#print(load_years)

from itertools import cycle

#repeat the date for 24 times for each hour in the day
year_2016 = load_years['2016'].repeat(24).reset_index(drop=True)
year_2011 = load_years['2011'].repeat(24).reset_index(drop=True)
#print(year_2016)

#use itertools.cycle to make the date repeat until it reaches the end of the dataframe
year_2016 = cycle(year_2016)
year_2011 = cycle(year_2011)

#create temporary data frames where they will iterate through. Needed to achieve the correct length of the DF
temp2016 = load_dur1.loc[load_dur1['R_Group'] == 'ERC'].copy()
temp2011 = load_dur1.loc[load_dur1['R_Group'] != 'ERC'].copy()

#iterate through the date until the end of DF 
temp2016['Date'] = [next(year_2016) for year in range(len(temp2016))]
temp2011['Date'] = [next(year_2011) for year in range(len(temp2011))]
#print(temp2016)
#print(temp2011)

load_dur2 = pd.concat([temp2016, temp2011], ignore_index=True)
#print(load_dur2)

#convert date to a datetime type 
load_dur2['Date'] = pd.to_datetime(load_dur2['Date'])
load_dur2['DOW'] = load_dur2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
load_dur2 = pd.merge(load_dur2,weekday,on='DOW',how='left')
#print(load_dur2)

load_dur3 = pd.merge(load_dur2,test4,on=['Region','R_Group','R_Subgroup','Season','Month','Day',
                                         'Hour','Hour_Counter','Load'],how='left')
load_dur3 = load_dur3.rename(columns={'Weekday':'Day_Type'})
#print(load_dur3)

#Return True if the load total equals the day identified as the max
load_dur3.loc[load_dur3['Day_Type'] == True, 'Day_Type'] = 'Weekday'
load_dur3.loc[load_dur3['Day_Type'] == False, 'Day_Type'] = 'Weekend'
load_dur3.loc[load_dur3['Load_Tot'] == load_dur3['Load_Max'], 'Day_Type'] = 'Peak'
load_dur3 = load_dur3.drop(['Load_Tot','Load_Max'], axis=1)
print(load_dur3)

          Region R_Group R_Subgroup  Season  Month  Day  Hour  Hour_Counter  \
0       ERC_REST     ERC       REST  winter      1    1     1             1   
1       ERC_REST     ERC       REST  winter      1    1     2             2   
2       ERC_REST     ERC       REST  winter      1    1     3             3   
3       ERC_REST     ERC       REST  winter      1    1     4             4   
4       ERC_REST     ERC       REST  winter      1    1     5             5   
...          ...     ...        ...     ...    ...  ...   ...           ...   
551875  WEC_SDGE     WEC       SDGE  winter     12  365    20          8756   
551876  WEC_SDGE     WEC       SDGE  winter     12  365    21          8757   
551877  WEC_SDGE     WEC       SDGE  winter     12  365    22          8758   
551878  WEC_SDGE     WEC       SDGE  winter     12  365    23          8759   
551879  WEC_SDGE     WEC       SDGE  winter     12  365    24          8760   

         Load       Date  DOW Day_Type  
0       34

# Case 1: Monthly, Weekend/weekday/peak day-types, 24 hours (864 segments)
#### Methodology: similar to two day type, just adding in peak day types to sort by

In [53]:
case1_load = load_dur3.copy()

aggregations = {'Load':['count',sum,'mean']}
case1 = case1_load.groupby(['Region','Month','Day_Type','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Day_Type','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
#case1.to_csv('outputs/load_segments_3daytype_monthly_24hr.csv')
print()

case1_load2 = pd.merge(case1_load,case1,on=['Region','Month','Day_Type','Hour'],how='left')
case1_load2 = case1_load2.sort_values(['Region','Load'])
print(case1_load2.head(3))
print('number of rows in dataset =',case1_load2.shape[0])
#case1_load2.to_csv('outputs/load_8760_3daytype_monthly_24hr.csv')

     Region  Month Day_Type  Hour  Hour_Tot  Load_Tot  Load_Avg
0  ERC_REST      1     Peak     1         1     38567   38567.0
1  ERC_REST      1     Peak     2         1     39107   39107.0
2  ERC_REST      1     Peak     3         1     40311   40311.0
3  ERC_REST      1     Peak     4         1     43115   43115.0
4  ERC_REST      1     Peak     5         1     47186   47186.0
number of rows in dataset = 54432

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   

       Load       Date  DOW Day_Type  Hour_Tot  Load_Tot   Load_Avg  
2042  26989 2016-03-26    5  Weekend         8    226231  28278.875  
2233  27006 2016-04-03    6  Weekend         8    228827  28603.375  
1706  27025 2016-03-12    5  Weekend         8    2262

# Case 2: Season, weekend/weekday/peak day-types, 24-hours (216 segments)

In [54]:
case2_load = load_dur3.copy()
case2_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case2_seasons = case2_seasons.drop(['bimonthly'], axis=1)
case2_seasons = case2_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case2_load = pd.merge(case2_load, case2_seasons, on='Month', how='left')
#print(case2_load)

aggregations = {'Load':['count',sum,'mean']}
case2 = case2_load.groupby(['Region','Season_Group','Day_Type','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season_Group','Day_Type','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
#case2.to_csv('outputs/load_segments_3daytype_season_24hr.csv')
print()

case2_load2 = pd.merge(case2_load,case2,on=['Region','Season_Group','Day_Type','Hour'],how='left')
case2_load2 = case2_load2.sort_values(['Region','Load'])
print(case1_load2.head(3))
print('number of rows in dataset =',case1_load2.shape[0])
#case2_load2.to_csv('outputs/load_8760_3daytype_season_24hr.csv')

     Region  Season_Group Day_Type  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST             1     Peak     1         3    121846  40615.333333
1  ERC_REST             1     Peak     2         3    123092  41030.666667
2  ERC_REST             1     Peak     3         3    126145  42048.333333
3  ERC_REST             1     Peak     4         3    133193  44397.666667
4  ERC_REST             1     Peak     5         3    144044  48014.666667
number of rows in dataset = 22680

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   

       Load       Date  DOW Day_Type  Hour_Tot  Load_Tot   Load_Avg  
2042  26989 2016-03-26    5  Weekend         8    226231  28278.875  
2233  27006 2016-04-03    6  Weekend         8    228827  

# Case 3: Annual, weekend/weekday/peak day-types, 24-hours (72 segments)

In [57]:
case3_load = load_dur3.copy()

aggregations = {'Load':['count',sum,'mean']}
case3 = case3_load.groupby(['Region','Day_Type','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Day_Type','Hour','Hour_Tot','Load_Tot','Load_Avg']
print(case3.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('outputs/load_segments_3daytype_annual_24hr.csv')
print()

case3_load2 = pd.merge(case3_load,case3,on=['Region','Day_Type','Hour'],how='left')
case3_load2 = case3_load2.sort_values(['Region','Load'])
print(case3_load2.head(3))
print('number of rows in dataset =',case3_load2.shape[0])
case3_load2.to_csv('outputs/load_8760_3daytype_annual_24hr.csv')

     Region Day_Type  Hour  Hour_Tot  Load_Tot      Load_Avg
0  ERC_REST     Peak     1        12    455893  37991.083333
1  ERC_REST     Peak     2        12    450804  37567.000000
2  ERC_REST     Peak     3        12    454384  37865.333333
3  ERC_REST     Peak     4        12    473371  39447.583333
4  ERC_REST     Peak     5        12    505999  42166.583333
number of rows in dataset = 4536

        Region R_Group R_Subgroup    Season  Month  Day  Hour  Hour_Counter  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3          2043   
2233  ERC_REST     ERC       REST  shoulder      4   94     2          2234   
1706  ERC_REST     ERC       REST  shoulder      3   72     3          1707   

       Load       Date  DOW Day_Type  Hour_Tot  Load_Tot      Load_Avg  
2042  26989 2016-03-26    5  Weekend       101   3270128  32377.504950  
2233  27006 2016-04-03    6  Weekend       101   3269439  32370.683168  
1706  27025 2016-03-12    5  Weekend       101   3270128  32377.